# Assignment: Segmenting and Clustering Neighborhoods in Toronto

## 1. Create Dataframe

### 1.1. Retrieve data

In [1]:
import pandas as pd

# Data source
urlWiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Read in data
torontoTable = pd.read_html(urlWiki)[0]

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
colNames = list(['PostalCode', 'Borough', 'Neighborhood'])
torontoTable.columns = colNames




### 1.2. Remove unassigned Boroughs

In [2]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#
# Count entries without assigned borough
cntNotAss = torontoTable[torontoTable['Borough']=='Not assigned']['Borough'].count()
# New dataframe: reset index to make it start at zero
torontoTableClean = torontoTable[torontoTable['Borough']!='Not assigned'].reset_index(drop=True)
# Check results
print('Entries without assigned borough:  {}'.format(cntNotAss))
print('Data size of original table:      {} lines'.format(torontoTable.shape[0]))
print('Data size of new table:           {} lines'.format(torontoTableClean.shape[0]))

Entries without assigned borough:  77
Data size of original table:      288 lines
Data size of new table:           211 lines


### 1.3. Name unassigned Neighbourhoods

In [3]:
# Find entries with unassigned Neighbourhoods
tmpIdx = torontoTableClean['Neighborhood']=='Not assigned'

# Before
torontoTableClean[tmpIdx]

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Not assigned


In [4]:
# Rename: it is only one entry. We could do it manually but this will work just fine.
torontoTableClean.loc[tmpIdx, 'Neighborhood'] = torontoTableClean.loc[tmpIdx, 'Borough']

In [5]:
# After
torontoTableClean[tmpIdx]

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Queen's Park


### 1.4. Combine Neighborhoods

In [6]:
# More than one neighborhood can exist in one postal code area.
# These rows will be combined into one row with the neighborhoods separated with a comma

# Get set of unique PostalCode entrie
uniquePostalCodes = torontoTableClean['PostalCode'].unique()
print('Number of unique postal codes: {}'.format(len(uniquePostalCodes)))

# New dataframe with combined neighbourhoods
torontoTableComb = pd.DataFrame(columns = colNames)

# Loop over unique postal codes
for postalCode in uniquePostalCodes:
    # Find all entries matching the current postal code
    tmpDf = torontoTableClean[torontoTableClean['PostalCode']==postalCode].reset_index(drop=True)
    # Loop over temporary table
    tmpStr=[]
    for idx, row in tmpDf.iterrows():
        # Combine neighbourhoods
        tmpStr.append(row.Neighborhood)
    # Create comma-separated string
    strNeighb = ', '.join(tmpStr)
    # Append new entry
    torontoTableComb = torontoTableComb.append({'PostalCode': postalCode, 'Borough': tmpDf.loc[0, 'Borough'], 'Neighborhood': strNeighb}, ignore_index=True)


Number of unique postal codes: 103


### 1.5. Dataframe size

In [7]:
print('Number of rows in combined dataframe: {}'.format(torontoTableComb.shape[0]))

Number of rows in combined dataframe: 103


## 2. Get latitude and longitude

### 2.1 Using geopy

In [8]:
# Unfortunately this will always fail for the same postal codes.
# I will use the provided CSV in the next cell

import numpy as np

# Use Nomimatim instead of geocoder
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="toronto_explorer")

# Arrays for latitude and longitude
arrLat = np.zeros(torontoTableComb.shape[0])
arrLon = np.zeros(torontoTableComb.shape[0])

# Loop over all rows and retrieve latitude and longitude for each postal code
for idx, row in torontoTableComb.iterrows():
    print(idx, end=" ")
    address = '{}, Toronto, Ontario, CA'.format(row.PostalCode)
    print(address, end= "")
    
    location = geolocator.geocode(address)
    
    if (location != None):
        arrLat[idx] = location.latitude
        arrLon[idx] = location.longitude
        print(', lat: {}, long: {}'.format(location.latitude, location.longitude))
    else:
        print(', None')


0 M3A, Toronto, Ontario, CA, lat: 43.653963, long: -79.387207
1 M4A, Toronto, Ontario, CA, None
2 M5A, Toronto, Ontario, CA

GeocoderTimedOut: Service timed out

### 2.2 Using CSV file

In [9]:
# Read in CSV
dfLatLong = pd.read_csv('https://cocl.us/Geospatial_data')

In [10]:
# Rename column for join
dfLatLong.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [11]:
# Merge dataframes
torontoTableComplete = torontoTableComb.merge(dfLatLong, on='PostalCode')

In [12]:
# Compare to sample of entries shown in assignment
torontoTableComplete[torontoTableComplete.isin({'PostalCode': ['M5G', 'M2H', 'M4B', 'M1J', 'M4G']})['PostalCode']]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
23,M4G,East York,Leaside,43.709060,-79.363452
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
27,M2H,North York,Hillcrest Village,43.803762,-79.363452
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476


### 2.3 Show dataframe

In [15]:
# Show entire dataframe
pd.set_option('display.max_colwidth', -1, 'display.max_rows', 103)
# Show
torontoTableComplete

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
